In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore

In [2]:
df1 = pd.read_csv("Atmospheric Data Precleaned.csv")

In [3]:
df1.head()

,Time,O3,O3_flag,NO2,NO2_flag,NO,NO_flag,CO,CO_flag,PM10,...,PM2.5_flag,Temp_Avg,RH_Avg,WSpeed_Avg,WSpeed_Max,WDir_Avg,WDir_SD,Rain_Tot,Press_Avg,Rad_Avg
0,2023-05-01 00:00:00,55.48,OK,0.72,OK,0.01,BDL,0.25,OK,25.47,...,OK,19.15,31.64,3.521,4.250,173.6,14.26,0.0,805.409,-1.521
1,2023-05-01 00:01:00,55.49,OK,0.81,OK,0.08,BDL,0.26,OK,25.74,...,OK,19.11,31.66,3.644,4.372,171.0,10.53,0.0,805.524,-1.521
2,2023-05-01 00:02:00,55.40,OK,0.93,OK,-0.05,BDL,0.27,OK,26.60,...,OK,19.09,31.70,4.139,5.253,178.6,15.72,0.0,805.436,-2.074
3,2023-05-01 00:03:00,55.20,OK,0.87,OK,0.03,BDL,0.28,OK,27.59,...,OK,19.10,31.76,3.692,4.925,186.1,17.43,0.0,805.450,-2.397
4,2023-05-01 00:04:00,55.41,OK,0.98,OK,-0.06,BDL,0.28,OK,27.83,...,OK,19.10,31.76,2.198,4.301,211.5,21.67,0.0,805.504,-1.706


In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408971 entries, 0 to 408970
Data columns (total 22 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Time        408971 non-null  object 
 1   O3          403172 non-null  float64
 2   O3_flag     408971 non-null  object 
 3   NO2         408725 non-null  float64
 4   NO2_flag    408971 non-null  object 
 5   NO          408725 non-null  float64
 6   NO_flag     408971 non-null  object 
 7   CO          408870 non-null  float64
 8   CO_flag     408971 non-null  object 
 9   PM10        408863 non-null  float64
 10  PM10_flag   408971 non-null  object 
 11  PM2.5       408860 non-null  float64
 12  PM2.5_flag  408971 non-null  object 
 13  Temp_Avg    408964 non-null  float64
 14  RH_Avg      408964 non-null  float64
 15  WSpeed_Avg  408964 non-null  float64
 16  WSpeed_Max  408964 non-null  float64
 17  WDir_Avg    408964 non-null  float64
 18  WDir_SD     408964 non-null  float64
 19  Ra

Checking atmospheric data flags

In [5]:
df1["O3_flag"].value_counts()

O3_flag
OK     401873
OS       5799
BDL      1282
OR         17
Name: count, dtype: int64

In [6]:
df1[(df1["O3_flag"] == "OS")]["O3"].value_counts()

Series([], Name: count, dtype: int64)

In [7]:
df1["NO2_flag"].value_counts()

NO2_flag
OK     406048
BDL      2616
OS        246
OR         61
Name: count, dtype: int64

In [8]:
df1["NO_flag"].value_counts()

NO_flag
BDL    317708
OK      90814
OS        246
OR        203
Name: count, dtype: int64

In [9]:
df1["CO_flag"].value_counts()

CO_flag
OK     402781
BDL      5981
OR        108
OS        101
Name: count, dtype: int64

In [10]:
df1["PM10_flag"].value_counts()

PM10_flag
OK     382706
BDL     23272
OR       2885
OS        108
Name: count, dtype: int64

In [11]:
df1["PM2.5_flag"].value_counts()

PM2.5_flag
OK     355963
BDL     44671
OR       8226
OS        111
Name: count, dtype: int64

We are going to eliminate null pollution data for our statistics analysis. So, we can drop those instances that has any flag out of service

In [12]:
df2 = df1.drop(df1[(df1["O3_flag"] == "OS") | (df1["NO2_flag"] == "OS") | (df1["NO_flag"] == "OS") | (df1["CO_flag"] == "OS") | (df1["PM10_flag"] == "OS") | (df1["PM2.5_flag"] == "OS")].index, axis=0)

In [13]:
df2.head()

,Time,O3,O3_flag,NO2,NO2_flag,NO,NO_flag,CO,CO_flag,PM10,...,PM2.5_flag,Temp_Avg,RH_Avg,WSpeed_Avg,WSpeed_Max,WDir_Avg,WDir_SD,Rain_Tot,Press_Avg,Rad_Avg
0,2023-05-01 00:00:00,55.48,OK,0.72,OK,0.01,BDL,0.25,OK,25.47,...,OK,19.15,31.64,3.521,4.250,173.6,14.26,0.0,805.409,-1.521
1,2023-05-01 00:01:00,55.49,OK,0.81,OK,0.08,BDL,0.26,OK,25.74,...,OK,19.11,31.66,3.644,4.372,171.0,10.53,0.0,805.524,-1.521
2,2023-05-01 00:02:00,55.40,OK,0.93,OK,-0.05,BDL,0.27,OK,26.60,...,OK,19.09,31.70,4.139,5.253,178.6,15.72,0.0,805.436,-2.074
3,2023-05-01 00:03:00,55.20,OK,0.87,OK,0.03,BDL,0.28,OK,27.59,...,OK,19.10,31.76,3.692,4.925,186.1,17.43,0.0,805.450,-2.397
4,2023-05-01 00:04:00,55.41,OK,0.98,OK,-0.06,BDL,0.28,OK,27.83,...,OK,19.10,31.76,2.198,4.301,211.5,21.67,0.0,805.504,-1.706


In [15]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 403017 entries, 0 to 408970
Data columns (total 22 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Time        403017 non-null  object 
 1   O3          403017 non-null  float64
 2   O3_flag     403017 non-null  object 
 3   NO2         403017 non-null  float64
 4   NO2_flag    403017 non-null  object 
 5   NO          403017 non-null  float64
 6   NO_flag     403017 non-null  object 
 7   CO          403017 non-null  float64
 8   CO_flag     403017 non-null  object 
 9   PM10        403017 non-null  float64
 10  PM10_flag   403017 non-null  object 
 11  PM2.5       403017 non-null  float64
 12  PM2.5_flag  403017 non-null  object 
 13  Temp_Avg    403010 non-null  float64
 14  RH_Avg      403010 non-null  float64
 15  WSpeed_Avg  403010 non-null  float64
 16  WSpeed_Max  403010 non-null  float64
 17  WDir_Avg    403010 non-null  float64
 18  WDir_SD     403010 non-null  float64
 19  Rain_To

In [16]:
df2["O3_flag"].value_counts()

O3_flag
OK     401719
BDL      1282
OR         16
Name: count, dtype: int64

In [17]:
df2["NO2_flag"].value_counts()

NO2_flag
OK     400340
BDL      2616
OR         61
Name: count, dtype: int64

In [18]:
df2["NO_flag"].value_counts()

NO_flag
BDL    313174
OK      89640
OR        203
Name: count, dtype: int64

In [19]:
df2["CO_flag"].value_counts()

CO_flag
OK     396928
BDL      5981
OR        108
Name: count, dtype: int64

In [20]:
df2["PM10_flag"].value_counts()

PM10_flag
OK     376860
BDL     23272
OR       2885
Name: count, dtype: int64

In [21]:
df2["PM2.5_flag"].value_counts()

PM2.5_flag
OK     350157
BDL     44634
OR       8226
Name: count, dtype: int64

Checking pollution data behind limit detection and span limit detection

| Pollutant | Analytical method | Detection limit | Span limit |
| - | - | - | - |
| O3 | UV absorption photometry | 0.03 ppb | 500 ppb |
| CO | Non-dispersive infrared | 0.04 ppm | 25 ppm |
| NO2, NO | Chemiluminescence | 0.4 ppb | 0.3, 0.5 ppm (NO2, NO) |
| PM2.5 | Separation by diameter and quantification by beta radiation attenuation | 4 ug/m^3 | 400 ug/m^3 |
| PM10 | Separation by diameter and quantification by beta radiation attenuation | 4 ug/m^3 | 400 ug/m^3 |

If any data exceeds the span limit, it must be null (as it may be an electronic or calibration outlier).

On the other hand, if data falls below the detection limit, we need to verify if it is within the positive and negative range of the detection limit value. If it falls within that range, we will change its value to half of the detection limit. If it falls outside the range, it must be null.

In [39]:
def O3_processing(data):
    if data > 500 or data < -0.03: return np.nan
    if data < 0.03 and data >= -0.03: return 0.03/2
    return data

def CO_processing(data):
    if data > 25 or data < -0.04: return np.nan
    if data < 0.04 and data >= -0.04: return 0.04/2
    return data

def NO2_processing(data):
    if data > 0.3 or data < -0.4: return np.nan
    if data < 0.4 and data >= -0.4: return 0.4/2
    return data

def NO_processing(data):
    if data > 0.5 or data < -0.4: return np.nan
    if data < 0.4 and data >= -0.4: return 0.4/2
    return data

def PM25_processing(data):
    if data > 400 or data < -4: return np.nan
    if data < 4 and data >= -4: return 4/2
    return data

def PM10_processing(data):
    if data > 400 or data < -4: return np.nan
    if data < 4 and data >= -4: return 4/2
    return data

In [40]:
df3 = df2.iloc[:,:]
df3["O3"] = df3["O3"].apply(O3_processing)
df3["CO"] = df3["CO"].apply(CO_processing)
df3["NO2"] = df3["NO2"].apply(NO2_processing)
df3["NO"] = df3["NO"].apply(NO_processing)
df3["PM2.5"] = df3["PM2.5"].apply(PM25_processing)
df3["PM10"] = df3["PM10"].apply(PM10_processing)

In [41]:
df2.head()

,Time,O3,O3_flag,NO2,NO2_flag,NO,NO_flag,CO,CO_flag,PM10,...,PM2.5_flag,Temp_Avg,RH_Avg,WSpeed_Avg,WSpeed_Max,WDir_Avg,WDir_SD,Rain_Tot,Press_Avg,Rad_Avg
0,2023-05-01 00:00:00,55.48,OK,0.72,OK,0.01,BDL,0.25,OK,25.47,...,OK,19.15,31.64,3.521,4.250,173.6,14.26,0.0,805.409,-1.521
1,2023-05-01 00:01:00,55.49,OK,0.81,OK,0.08,BDL,0.26,OK,25.74,...,OK,19.11,31.66,3.644,4.372,171.0,10.53,0.0,805.524,-1.521
2,2023-05-01 00:02:00,55.40,OK,0.93,OK,-0.05,BDL,0.27,OK,26.60,...,OK,19.09,31.70,4.139,5.253,178.6,15.72,0.0,805.436,-2.074
3,2023-05-01 00:03:00,55.20,OK,0.87,OK,0.03,BDL,0.28,OK,27.59,...,OK,19.10,31.76,3.692,4.925,186.1,17.43,0.0,805.450,-2.397
4,2023-05-01 00:04:00,55.41,OK,0.98,OK,-0.06,BDL,0.28,OK,27.83,...,OK,19.10,31.76,2.198,4.301,211.5,21.67,0.0,805.504,-1.706


In [42]:
df3.head()

,Time,O3,O3_flag,NO2,NO2_flag,NO,NO_flag,CO,CO_flag,PM10,...,PM2.5_flag,Temp_Avg,RH_Avg,WSpeed_Avg,WSpeed_Max,WDir_Avg,WDir_SD,Rain_Tot,Press_Avg,Rad_Avg
0,2023-05-01 00:00:00,55.48,OK,NaN,OK,0.2,BDL,0.25,OK,25.47,...,OK,19.15,31.64,3.521,4.250,173.6,14.26,0.0,805.409,-1.521
1,2023-05-01 00:01:00,55.49,OK,NaN,OK,0.2,BDL,0.26,OK,25.74,...,OK,19.11,31.66,3.644,4.372,171.0,10.53,0.0,805.524,-1.521
2,2023-05-01 00:02:00,55.40,OK,NaN,OK,0.2,BDL,0.27,OK,26.60,...,OK,19.09,31.70,4.139,5.253,178.6,15.72,0.0,805.436,-2.074
3,2023-05-01 00:03:00,55.20,OK,NaN,OK,0.2,BDL,0.28,OK,27.59,...,OK,19.10,31.76,3.692,4.925,186.1,17.43,0.0,805.450,-2.397
4,2023-05-01 00:04:00,55.41,OK,NaN,OK,0.2,BDL,0.28,OK,27.83,...,OK,19.10,31.76,2.198,4.301,211.5,21.67,0.0,805.504,-1.706


In [43]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 403017 entries, 0 to 408970
Data columns (total 22 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Time        403017 non-null  object 
 1   O3          402971 non-null  float64
 2   O3_flag     403017 non-null  object 
 3   NO2         1769 non-null    float64
 4   NO2_flag    403017 non-null  object 
 5   NO          321337 non-null  float64
 6   NO_flag     403017 non-null  object 
 7   CO          402909 non-null  float64
 8   CO_flag     403017 non-null  object 
 9   PM10        400054 non-null  float64
 10  PM10_flag   403017 non-null  object 
 11  PM2.5       394790 non-null  float64
 12  PM2.5_flag  403017 non-null  object 
 13  Temp_Avg    403010 non-null  float64
 14  RH_Avg      403010 non-null  float64
 15  WSpeed_Avg  403010 non-null  float64
 16  WSpeed_Max  403010 non-null  float64
 17  WDir_Avg    403010 non-null  float64
 18  WDir_SD     403010 non-null  float64
 19  Rain_To

In [32]:
df3[~(df3["NO2"].isnull())]

,Time,O3,O3_flag,NO2,NO2_flag,NO,NO_flag,CO,CO_flag,PM10,...,PM2.5_flag,Temp_Avg,RH_Avg,WSpeed_Avg,WSpeed_Max,WDir_Avg,WDir_SD,Rain_Tot,Press_Avg,Rad_Avg
974,2023-05-01 16:14:00,47.64,OK,0.2,BDL,0.2,BDL,0.15,OK,30.31,...,OK,26.30,15.42,3.791,4.905,225.6,18.010,0.0,802.478,698.600
979,2023-05-01 16:19:00,47.10,OK,0.2,BDL,0.2,BDL,0.16,OK,29.62,...,OK,26.84,15.51,3.166,3.912,170.5,25.520,0.0,802.363,667.100
1035,2023-05-01 17:15:00,46.65,OK,0.2,BDL,0.2,BDL,0.16,OK,22.21,...,OK,26.22,16.51,6.852,7.875,164.8,13.360,0.0,802.227,424.500
1037,2023-05-01 17:17:00,47.98,OK,0.2,BDL,0.2,BDL,0.14,OK,22.79,...,OK,25.84,16.71,6.166,8.310,164.1,9.160,0.0,802.356,415.800
1042,2023-05-01 17:22:00,46.85,OK,0.2,BDL,0.2,BDL,0.15,OK,23.76,...,OK,26.11,16.48,7.456,7.885,165.3,7.617,0.0,802.410,393.700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408490,2024-03-13 20:00:00,42.68,OK,0.2,BDL,0.2,BDL,0.23,OK,27.82,...,OK,21.39,39.85,2.427,3.082,204.4,17.610,0.0,803.841,-1.613
408492,2024-03-13 20:02:00,43.22,OK,0.2,BDL,0.2,BDL,0.22,OK,29.48,...,OK,21.17,40.06,0.341,1.075,222.2,15.150,0.0,803.795,-1.060
408493,2024-03-13 20:03:00,43.14,OK,0.2,BDL,0.2,BDL,0.23,OK,29.55,...,OK,21.11,40.09,0.913,1.188,235.4,23.390,0.0,803.871,-1.014
408547,2024-03-13 20:57:00,42.90,OK,0.2,BDL,0.2,BDL,0.24,OK,30.25,...,OK,20.73,41.47,2.889,3.727,171.1,14.980,0.0,804.368,-1.890


Checking meteorological variables

In [56]:
df3["Temp_Avg"].describe()

count    403010.000000
mean         18.023412
std           5.174202
min           2.966000
25%          14.650000
50%          17.730000
75%          21.710000
max          33.570000
Name: Temp_Avg, dtype: float64

In [57]:
df3["RH_Avg"].describe()

count    403010.000000
mean         63.074023
std          22.334623
min           7.406000
25%          46.170000
50%          64.680000
75%          82.300000
max          99.900000
Name: RH_Avg, dtype: float64

In [68]:
df3["WSpeed_Avg"].describe()

count    403010.000000
mean          1.516865
std           1.492161
min           0.000000
25%           0.145000
50%           1.172000
75%           2.386000
max          17.150000
Name: WSpeed_Avg, dtype: float64

In [69]:
df3["WSpeed_Max"].describe()

count    403010.000000
mean          1.994164
std           1.885134
min           0.000000
25%           0.410000
50%           1.556000
75%           3.072000
max          18.780000
Name: WSpeed_Max, dtype: float64

In [70]:
df3["WDir_Avg"].describe()

count    403010.000000
mean        142.570308
std         107.842669
min           0.000000
25%          40.160000
50%         143.900000
75%         225.700000
max         360.000000
Name: WDir_Avg, dtype: float64

In [71]:
df3["WDir_SD"].describe()

count    403010.000000
mean          9.343657
std          10.090872
min           0.000000
25%           0.076000
50%           7.467000
75%          14.010000
max         102.700000
Name: WDir_SD, dtype: float64

In [72]:
df3["Rain_Tot"].describe()

count    403010.000000
mean          0.001136
std           0.020003
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.800000
Name: Rain_Tot, dtype: float64

In [76]:
df3["Press_Avg"].describe()

count    403010.000000
mean        805.890962
std           1.843219
min         797.895000
25%         804.696000
50%         805.931000
75%         807.145000
max         812.652000
Name: Press_Avg, dtype: float64

In [77]:
df3["Rad_Avg"].describe()

count    403004.000000
mean        241.493380
std         350.539892
min         -16.090000
25%          -1.198000
50%           3.365000
75%         422.200000
max        1439.000000
Name: Rad_Avg, dtype: float64

It seems like the only variable that must be cleaned is Rad_Avg

**Radiation (Rad_Avg)**

A simple filter is used for the Rad_Avg column: values greater than 0.001 are left in the column,
and the rest are replaced by null.

In [78]:
def rad_processing(data):
    if data < 0.001: return np.nan
    return data

In [79]:
df4 = df3.iloc[:,:]
df4["Rad_Avg"] = df4["Rad_Avg"].apply(rad_processing)

In [80]:
df4["Rad_Avg"].describe()

count    211218.000000
mean        461.952268
std         363.761674
min           0.046000
25%         124.700000
50%         390.300000
75%         790.800000
max        1439.000000
Name: Rad_Avg, dtype: float64

In [81]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 403017 entries, 0 to 408970
Data columns (total 22 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Time        403017 non-null  object 
 1   O3          402971 non-null  float64
 2   O3_flag     403017 non-null  object 
 3   NO2         1769 non-null    float64
 4   NO2_flag    403017 non-null  object 
 5   NO          321337 non-null  float64
 6   NO_flag     403017 non-null  object 
 7   CO          402909 non-null  float64
 8   CO_flag     403017 non-null  object 
 9   PM10        400054 non-null  float64
 10  PM10_flag   403017 non-null  object 
 11  PM2.5       394790 non-null  float64
 12  PM2.5_flag  403017 non-null  object 
 13  Temp_Avg    403010 non-null  float64
 14  RH_Avg      403010 non-null  float64
 15  WSpeed_Avg  403010 non-null  float64
 16  WSpeed_Max  403010 non-null  float64
 17  WDir_Avg    403010 non-null  float64
 18  WDir_SD     403010 non-null  float64
 19  Rain_To

In [82]:
df4.to_csv("Atmospheric Data Cleaned.csv")